# Formeln vom Manual

In [3]:
import numpy as np
import uncertainties as u
import os
import pint # pip install pint OR conda install pint
units = pint.UnitRegistry()

# Formel (2)
# n'2(nu) = 1 + A/(nu0dash^2-nu^2)
# nu0dash, A sind Materialkonstanten
def n_squared_of_nu(nu, nu0dash, A):
    n_sq = 1 + A/(nu0dash**2-nu**2)
    return n_sq

# Formel (2) und die Wurzel gezogen
def n_of_nu(nu, nu0dash, A):
    n = np.sqrt(n_squared_of_nu(nu, nu0dash,A))
    return n

# First Task: Determine $\varphi$
Measure both positions where the reflections of the slit image are observed: $A_{1}$, $A_{2}$. Then $\beta = A_{1} - A_{2}$. Make sure the signs are correct (can we take the absolute value?). Do this three times, so we get three $\beta$ values. Take the mean of $\beta$. Then $\varphi = \pi - \beta$.

In [ ]:
A1_meas, A2_meas = np.genfromtxt(os.path.join(os.getcwd(), "task1_measure_phi_data.csv"), delimiter=',', skip_header=1, unpack=True)

A_meas_uncertainty_guess = 0.000 # TODO change this value
A1 = [u.ufloat(a1, A_meas_uncertainty_guess) for a1 in A1_meas] * units.mm # TODO may have to change units if not mm was measured
A2 = [u.ufloat(a2, A_meas_uncertainty_guess) for a2 in A2_meas] * units.mm # TODO may have to change units if not mm was measured

